In [215]:
import numpy as np
import math

In [216]:
def step(x, lamb, gradf, hessianf, h, gradh, hessianh, stepsize=1):
    """
    all functions should return numpy arrays of a proper size
    :param x: np.array of size
    :param lamb: np.array of size number of constraints (nc) x 1
    :param gradf: callable returning np.array of size number of arguments (na) x 1
    :param hessianf: callable returning np.array of size na x na
    :param h: callable returning np.array of size nc x 1
    :param gradh: np.array of size callable na x nc
    :param hessianh: np.array of size callable
    :param stepsize: optinial float for the stepsize
    :return: x: one step towards optimal solution of quadratic approximation
    """

    hessianLagrangian = hessianf(x) - lamb*hessianh(x)
    gradh = gradh(x).reshape((gradh(x).shape[0],1))


    topPart = np.concatenate((hessianLagrangian, -gradh), axis=1)
    bottomPart = np.concatenate((-gradh.T, np.zeros((gradh.shape[1], gradh.shape[1]))), axis=1)

    mat = np.concatenate((topPart, bottomPart))
    rhs = np.concatenate((-(gradf(x)-lamb*gradh.T), h(x)), axis=1).reshape((3,1))
    try:
        res = np.linalg.solve(mat, rhs) # np.linalg.inv(mat) @ rhs #

        xdir = res[0:len(res)-1].reshape(x.shape)
        lambdir = res[-1]
        return x + xdir, lamb + lambdir
    except:
        return x, lamb



In [217]:
def gradf(x):
    return np.array([3*math.exp(3*x[0]), -4*math.exp(-4*x[1])])

def gradh(x):
    return np.array([2*x[0],2*x[1]])

def h(x):
    return np.array([x[0]**2 + x[1]**2 - 1]).reshape((1,1))

def hessianh(x):
    return np.array([2,0,0,2]).reshape(2,2)

def hessianf(x):
    return np.array([9*math.exp(3*x[0]),0,0,16*math.exp(-4*x[1])]).reshape((2,2))


In [218]:
# Try our Step function with the given example

x, l = step(np.array([-1, 1]), -1, gradf, hessianf, h, gradh, hessianh)
print(x,l)

[-0.77422564  0.72577436] [-0.35103786]


In [219]:
def optimise(x_start, lamb_start, gradf, hessianf, h, gradh, hessianh, steps=100, stepsize=1):
    x, lamb = x_start, lamb_start
    for i in range(steps):
        #some error handling to get running code
        try:
            x, lamb = step(x, lamb, gradf, hessianf, h, gradh, hessianh)
        except OverflowError:
            break
    return x, lamb

In [220]:
x = np.array([50, 100])
lamb = 5
steps = 100
print(optimise(x, lamb, gradf, hessianf, h, gradh, hessianh, steps))


(array([-0.74833549,  0.66332043]), array([-0.21232494]))


In [221]:
intuitiveX = [[0,0], [0.05,0.05], [-1,1], [-10,10], [-100, 100], [1,-1], [10,-10], [100, -100], [1,1], [1.8, 1.8], [1.85, 1.85], [5,5], [-1,-1], [-1.8, -1.8], [-1.85, -1.85], [-5, -5]] #some intuitively choosen startpoints
randomX = np.random.uniform(-100, 100, (20,2)) #some random points
startX = np.append(randomX, startX, axis=0)
startL = [-10, -5, -1, -0.1, 0, 0.1, 1, 5, 10]
startingPoints = [[-np.array(x), l] for x in startX for l in startL]
steps = 500
x_optimal = np.array([-0.74834, 0.66332])
lamb_optimal = -0.21233


for x,lamb in startingPoints:
    reached = "     failed     "
    x_reach, lamb_reach = optimise(x, lamb, gradf, hessianf, h, gradh, hessianh, steps)
    if np.linalg.norm(x_reach-x_optimal) < 1e-5:
        reached = "'''successful'''"
        if np.linalg.norm(lamb_reach-lamb_optimal) < 1e-4:
            reached = "|||super successful|||"
    print(f"{reached}.  after {steps} iterations we got from {x} with initial lambda {lamb} to {x_reach, lamb_reach}")


     failed     .  after 500 iterations we got from [ 94.9548048  -47.39042384] with initial lambda -10 to (array([520.25057132, -76.24784074]), array([-9.60704442e+124]))
     failed     .  after 500 iterations we got from [ 94.9548048  -47.39042384] with initial lambda -5 to (array([520.25057132, -76.24784074]), array([-9.60704442e+124]))
     failed     .  after 500 iterations we got from [ 94.9548048  -47.39042384] with initial lambda -1 to (array([520.25057132, -76.24784074]), array([-9.60704442e+124]))
     failed     .  after 500 iterations we got from [ 94.9548048  -47.39042384] with initial lambda -0.1 to (array([520.25057132, -76.24784074]), array([-9.60704442e+124]))
     failed     .  after 500 iterations we got from [ 94.9548048  -47.39042384] with initial lambda 0 to (array([520.25057132, -76.24784074]), array([-9.60704442e+124]))
     failed     .  after 500 iterations we got from [ 94.9548048  -47.39042384] with initial lambda 0.1 to (array([520.25057132, -76.24784074])

What we can observe is, that with starting points,r that have the same signs as the optimal point the newton like approach has no problem to converge to the optimal solution.
If the signs are changed the behaviour changes drastically (moe experiments on that?).
For starting points with the same sign it seems like we converge until somewhere in the interval $\pm [0.5, 1.85]$
The choice of the initial lambda seems to only matter in the border region of the convergence area, but I cant really determine a pattern in that.
To conclude: Minus Plus is handled well, Plus Minus is okay, same sign is only in some region well handled

In [222]:
#for comparision
for x in startX:
    reached = "     failed     "
    x_reach, lamb_reach = optimise(x, -1, gradf, hessianf, h, gradh, hessianh, steps)
    if np.linalg.norm(x_reach-x_optimal) < 1e-5:
        reached = "'''successful'''"
        if np.linalg.norm(lamb_reach-lamb_optimal) < 1e-4:
            reached = "|||super successful|||"
    print(f"{reached}.  after {steps} iterations we got from {x} with initial lambda {lamb} to {x_reach, lamb_reach}.  Distance:{np.linalg.norm(x_reach-x_optimal)}")

|||super successful|||.  after 500 iterations we got from [-94.9548048   47.39042384] with initial lambda 10 to (array([-0.74833549,  0.66332043]), array([-0.21232494])).  Distance:4.534001547404924e-06
|||super successful|||.  after 500 iterations we got from [-17.29693612  99.71748789] with initial lambda 10 to (array([-0.74833549,  0.66332043]), array([-0.21232494])).  Distance:4.53400154739428e-06
     failed     .  after 500 iterations we got from [ 73.81702944 -99.71831704] with initial lambda 10 to (array([16640.37592936,  -148.19801526]), array([-4.01659751e+197])).  Distance:16641.79006733828
     failed     .  after 500 iterations we got from [15.06536526 25.72759068] with initial lambda 10 to (array([ 0.91041323, -0.41370006]), array([25.2938552])).  Distance:1.9777346917460152
     failed     .  after 500 iterations we got from [-31.12248008 -15.17948328] with initial lambda 10 to (array([2097.4506112 ,  -13.67948328]), array([-1.03746171e+10])).  Distance:2098.247972676499

With a constant lambda we can see, that espacially for the randomly generated points the Newton like method performs poorly as expected.

In [223]:
def grd_M(x):
    g= np.array([3*math.e**(3*x[0])+40*x[0]**3+40*x[0]*x[1]**2-40*x[0], -4*math.e**(-4*x[1])+40*x[1]**3+40*x[1]*x[0]**2-40*x[1]])
    return g

def grad_desc(x,a=0.01):
    g = grd_M(x)
    normed = g/np.linalg.norm(g)
    return x-a*normed


def meritOptimise(start, grd_M, steps, step=0.01):
    x = start
    for c in range(steps):
        x = grad_desc(x, step)
        if np.linalg.norm(grd_M(x)) < 1e-2:
            print(f"exited after {c} iterations with gradient {grd_M(x)}")
            break

    return x

# test
for x in startX:
    reached = "     failed     "
    x_reach = meritOptimise(x, grd_M, 50000)
    if np.linalg.norm(x_reach-x_optimal) < 5e-2:
        reached = "'''successful'''"
        if np.linalg.norm(x_reach-x_optimal) < 1e-3:
            reached = "|||super successful|||"
    print(f"{reached}.  We got from {x} to {x_reach}. Distance:{np.linalg.norm(x_reach-x_optimal)}")

'''successful'''.  We got from [-94.9548048   47.39042384] to [-0.75614023  0.66984314]. Distance:0.010168329140233042
'''successful'''.  We got from [-17.29693612  99.71748789] to [-0.74629364  0.66109946]. Distance:0.0030196630887491106
     failed     .  We got from [ 73.81702944 -99.71831704] to [ 73.81702944 -99.71831704]. Distance:125.04586108756864
'''successful'''.  We got from [15.06536526 25.72759068] to [-0.75413029  0.66805723]. Distance:0.007481227841370433
'''successful'''.  We got from [-31.12248008 -15.17948328] to [-0.75837757  0.67182705]. Distance:0.01315761106814126
'''successful'''.  We got from [  2.99568841 -62.27089315] to [-0.75787377  0.67138069]. Distance:0.012484694266286487
'''successful'''.  We got from [-52.26291033 -43.25116112] to [-0.75118393  0.66544072]. Distance:0.00354759464824294
'''successful'''.  We got from [ 97.16678649 -38.85286956] to [-0.74966212  0.66409194]. Distance:0.0015309769313803507
'''successful'''.  We got from [-69.65826692 -77.6

As we can see, we are getting pretty close on a lot(more then the Newton like Method before) of solutions and on some of them even pretty close. But in general not as close as with the Newton Method.

Let's combine both methods and use the strength of both

In [224]:
def combinedMethod(x_start, lamb_start, rho, gradf, hessianf, h, gradh, hessianh, steps_Newton=100, steps_Merit=5000, stepsize=0.1):
    grd_M = lambda x: gradf(x) + 2*rho*gradh(x)
    x_merit = meritOptimise(x_start, grd_M, steps_Merit, stepsize)

    x_end = optimise(x_merit, lamb_start, gradf, hessianf, h, gradh, hessianh, steps_Newton)
    return x_merit, x_end

#test how far can we go
x_1, x_2 = combinedMethod(np.array([120, 120]), 5, 10, gradf, hessianf, h, gradh, hessianh)
print(x_1, x_2)

[120. 120.] (array([   119.33333333, -21095.13989197]), array([182014.33146124]))


In [225]:
#test on all startpoints
for x in startX:
    reached = "     failed     "
    x_merit, [x_reach, lamb_reach] = combinedMethod(x, 1, 10, gradf, hessianf, h, gradh, hessianh)
    print(x_reach)
    if np.linalg.norm(x_reach-x_optimal) < 5e-5:
        reached = "'''successful'''"
        if np.linalg.norm(x_reach-x_optimal) < 1e-5:
            reached = "|||super successful|||"
    print(f"{reached}.  We got from {x} to {x_reach}.  Distance:{np.linalg.norm(x_reach-x_optimal)}")


[-0.74833549  0.66332043]
|||super successful|||.  We got from [-94.9548048   47.39042384] to [-0.74833549  0.66332043].  Distance:4.534001547404924e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [-17.29693612  99.71748789] to [-0.74833549  0.66332043].  Distance:4.534001547515435e-06
[16640.37592936  -148.19801526]
     failed     .  We got from [ 73.81702944 -99.71831704] to [16640.37592936  -148.19801526].  Distance:16641.79006733828
[-0.74833549  0.66332043]
|||super successful|||.  We got from [15.06536526 25.72759068] to [-0.74833549  0.66332043].  Distance:4.53400154739428e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [-31.12248008 -15.17948328] to [-0.74833549  0.66332043].  Distance:4.53400154739428e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [  2.99568841 -62.27089315] to [-0.74833549  0.66332043].  Distance:4.534001547515435e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [-52.26291033 -43.251

As expected combining both methods leads to even better results, because we reach the proximity of the solution by minimizing the merit function and can converge to the optimal solution very fast afterwards by using the Newton-like Method.
But there are some points which neither of the methods can't handle, for example $x_0=(100, -100)$
